References

- https://www.kaggle.com/code/richolson/ai-math-olympiad-qwen2-5-72b for showing how to submit
- https://www.kaggle.com/code/abdullahmeda/load-72b-awq-model-using-vllm-on-l4-x4
- https://www.kaggle.com/code/huikang/qwen2-5-math-1-5b-instruct

In [ ]:
import os  # operating system
import time
import warnings
import re  # regular expressions

import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl  # polars

import torch  # pytorch
import kaggle_evaluation.aimo_2_inference_server

pd.set_option('display.max_colwidth', None)  # display all columns
cutoff_time = time.time() + (4 * 60 + 30) * 60  # 4 hours 30 minutes from now

In [ ]:
import json
from datetime import datetime


def log_model_interaction(messages,
                          response,
                          log_file="model_interactions.jsonl"):
    """
    Log model interactions to a JSONL file.
    
    Args:
        messages: List of message dictionaries containing the input prompt
        response: Model's response
        log_file: Path to the log file
    """
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "input_messages": messages,
        "model_response": response
    }

    # Write to JSONL file
    with open(log_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(log_entry, ensure_ascii=False) + '\n')

In [2]:
# VLLM model and sampling parameters for inference and evaluation
from vllm import LLM, SamplingParams

# ignore warnings
warnings.simplefilter('ignore')

# use all GPUs available on the Kaggle notebook (4 GPUs) for tensor parallelism with PyTorch and VLLM model inference
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
# disable tokenizers parallelism to avoid deadlocks with PyTorch tensor parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"

2024-11-16 00:04:22,411	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
# Get number of available GPUs for tensor parallelism
num_gpus = torch.cuda.device_count()

llm_model_pth = '/kaggle/input/qwen_qwen2.5_math_72bawq2/transformers/72b/3'

# Load the VLLM model for inference
llm = LLM(
    llm_model_pth,
    # The data type for the model weights and activations. Options: "float", "half". Default is "float"
    dtype="half",
    # Maximum number of sequences per iteration. Default is 256 for "float" and 128 for "half"
    max_num_seqs=16,  # Increased from 8 to 16
    # Model context length. Default is 4096
    max_model_len=4096,
    # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer. Default is False
    trust_remote_code=True,
    # The number of GPUs to use for distributed execution with tensor parallelism. Default is 4
    tensor_parallel_size=4,
    # Use half of the available GPUs for swap space
    swap_space=max(1, num_gpus // 2),
    # The ratio (between 0 and 1) of GPU memory to reserve for the model. Default is 0.95
    gpu_memory_utilization=0.97,  # Increased from 0.95 to 0.97
    # Random seed for reproducibility. Default is 2024
    seed=2024,
    # Enforce eager execution for debugging. Default is False
    enforce_eager=True,
)

WARNING 11-16 00:04:27 config.py:1668] Casting torch.bfloat16 to torch.float16.
INFO 11-16 00:04:54 awq_marlin.py:97] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 11-16 00:04:54 config.py:905] Defaulting to use mp for distributed inference
INFO 11-16 00:04:54 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/kaggle/input/qwen_qwen2.5_math_72bawq2/transformers/72b/3', speculative_config=None, tokenizer='/kaggle/input/qwen_qwen2.5_math_72bawq2/transformers/72b/3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cud

Loading safetensors checkpoint shards:   0% Completed | 0/9 [00:00<?, ?it/s]


(VllmWorkerProcess pid=357) INFO 11-16 00:10:10 model_runner.py:1067] Loading model weights took 9.7836 GB
INFO 11-16 00:10:10 model_runner.py:1067] Loading model weights took 9.7836 GB
(VllmWorkerProcess pid=356) INFO 11-16 00:10:10 model_runner.py:1067] Loading model weights took 9.7836 GB
(VllmWorkerProcess pid=355) INFO 11-16 00:10:10 model_runner.py:1067] Loading model weights took 9.7836 GB
INFO 11-16 00:10:17 distributed_gpu_executor.py:57] # GPU blocks: 8638, # CPU blocks: 3276
INFO 11-16 00:10:17 distributed_gpu_executor.py:61] Maximum concurrency for 4096 tokens per request: 33.74x
INFO 11-16 00:10:21 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-16 00:10:21 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_me

In [4]:
# Get the tokenizer for the VLLM model
tokenizer = llm.get_tokenizer()

In [ ]:
def extract_python_code(text):
    """
    Extracts Python code blocks from markdown text and combines them.
    
    Finds all text between ```python and ``` markers (markdown code blocks),
    and joins multiple code blocks with double newlines.
    
    Args:
        text (str): Input text containing markdown-style Python code blocks
        
    Returns:
        str: Concatenated Python code blocks, separated by double newlines
        
    Example:
        Input:  '''
                ```python
                x = 1
                ```
                ```python
                y = 2
                ```
                '''
        Output: "x = 1\n\ny = 2"
    """
    # Regular expression pattern for Python code blocks
    pattern = r'```python\s*(.*?)\s*```'
    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text, re.DOTALL)
    # Return the Python code blocks as a string separated by two newlines
    return "\n\n".join(matches)  # Join all matches with two newlines

In [ ]:
import keyword  # Python keywords


def process_python_code(query):
    """
    Preprocesses Python code by adding debug capabilities for mathematical problem solving.
    
    This function takes Python code (typically generated by an AI solving math problems) 
    and enhances it by:
    1. Adding common math-related imports (math, numpy, sympy)
    2. Adding automatic print statements for every top-level variable assignment
    
    Args:
        query (str): Original Python code
        
    Returns:
        str: Processed code with added imports and debug print statements
    
    Example:
        Input:  "x = 42"
        Output: "import math\\nimport numpy as np\\nimport sympy as sp\\n
                 x = 42\\n
                 try:\\n    print(f\\"x={str(x)[:100]}\\")\\nexcept:\\n    pass\\n"
    """
    # Add import statements
    # Also print variables if they are not inside any indentation
    query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
    # Split the query into rows
    current_rows = query.strip().split("\n")
    new_rows = []
    for row in current_rows:
        # Add the current row
        new_rows.append(row)
        if not row.startswith(" ") and "=" in row:
            # Get the variable name
            variables_to_print = row.split("=")[0].strip()
            # Split multiple variables
            for variable_to_print in variables_to_print.split(","):
                # Remove leading/trailing spaces
                variable_to_print = variable_to_print.strip()
                # Check if the variable is a valid identifier and not a Python keyword
                if variable_to_print.isidentifier(
                ) and not keyword.iskeyword(variable_to_print):
                    if row.count("(") == row.count(")") and row.count(
                            "[") == row.count("]"):
                        # TODO: use some AST to parse code
                        new_rows.append(
                            f'\ntry:\n    print(f"{variable_to_print}={{str({variable_to_print})[:100]}}")\nexcept:\n    pass\n'
                        )
    return "\n".join(new_rows)

In [ ]:
# Extract text inside "boxed" curly braces {text} using regular expressions (regex) and return it as a string or
# an empty string if no match is found (e.g., "{text}" will return "text")
def extract_boxed_text(text):
    # Regular expression pattern for boxed text
    pattern = r'oxed{(.*?)}'
    # Find all matches of the pattern in the text
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    # Return the first match
    return matches[0]


from collections import Counter  # Counter for counting occurrences of elements in a list
import random  # random numbers and shuffling lists (e.g., for random sampling)


# Select the most common answer from a list of answers and return it as an integer (e.g., [1, 2, 2, 3, 3, 3] will return 3)
def select_answer(answers):
    # Counter for counting occurrences of elements
    counter = Counter()
    for answer in answers:
        try:
            # Check if the answer is an integer
            if int(answer) == float(answer):
                # Add the answer to the counter with a small random noise to break ties
                counter[int(answer)] += 1 + random.random() / 1_000
        except:
            pass
    if not counter:
        # Return the default answer if no valid answers are found
        return 210
    # Select the most common answer from the counter
    _, answer = sorted([(v, k) for k, v in counter.items()], reverse=True)[0]
    # Return the answer modulo 1000 (e.g., 1000 will be returned as 0)
    # TODO: print the answer before modulo 1000 to check if it is correct
    return answer % 1000

In [ ]:
import os
import tempfile  # temporary files and directories
import subprocess  # subprocesses


# Python Read-Eval-Print Loop (REPL) for executing Python code with a timeout using subprocesses and temporary 
# files and directories for security and resource management
class PythonREPL:

    def __init__(self, timeout=5):
        # Timeout for code execution in seconds
        self.timeout = timeout

    def __call__(self, query):
        # Create a temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            # Temporary Python file path
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            # Write the query to the temporary file
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)

            try:
                # Execute the Python file with a timeout
                result = subprocess.run(
                    ["python3", temp_file_path],
                    # Capture stdout and stderr
                    capture_output=True,
                    # Do not raise an exception on non-zero exit codes
                    check=False,
                    # Return stdout and stderr as text
                    text=True,
                    # Timeout for code execution
                    timeout=self.timeout,
                )
            except subprocess.TimeoutExpired:
                return False, f"Execution timed out after {self.timeout} seconds."

            stdout = result.stdout.strip()
            stderr = result.stderr.strip()

            if result.returncode == 0:
                return True, stdout
            else:
                # Process the error message to remove the temporary file path
                # This makes the error message cleaner and more user-friendly
                error_lines = stderr.split("\n")
                cleaned_errors = []
                for line in error_lines:
                    if temp_file_path in line:
                        # Remove the path from the error line
                        line = line.replace(temp_file_path, "<temporary_file>")
                    cleaned_errors.append(line)
                cleaned_error_msg = "\n".join(cleaned_errors)
                # Include stdout in the error case
                combined_output = f"{stdout}\n{cleaned_error_msg}" if stdout else cleaned_error_msg
                return False, combined_output

In [ ]:
# Sampling parameters for the VLLM model inference and evaluation (e.g., temperature, min_p, max_tokens, etc.)
sampling_params = SamplingParams(
    # The temperature of the sampling distribution. Higher values mean more randomness. Default is 1.0
    temperature=0.7,  # Changed from 1.0
    # The minimum token probability for nucleus sampling. Default is 0.01
    # min_p=0.01,
    # The maximum token probability for nucleus sampling. Default is 1.0
    top_p=0.8,
    # Whether to skip special tokens in the output. Default is True
    #skip_special_tokens=True,
    # Maximum number of tokens to generate. Default is 1024
    max_tokens=3072,  # Changed from 2400
    # Stop generation at the end of the code block
    stop=["```\n"],
    # Include the stop string in the output. Default is False
    include_stop_str_in_output=True,
)


# Generate a message using the VLLM model and return the generated message as a string (e.g., "Hello, world!")
# or an empty string if no message is generated (e.g., if the input is empty)
def batch_message_generate(list_of_messages) -> list[list[dict]]:

    list_of_texts = [
        # Apply the chat template to each conversation and add the generation prompt to each message in the conversation
        # (e.g., "role: user\ncontent: Hello, world!") and return the list of texts as a list of strings
        # (e.g., ["role: user\ncontent: Hello, world!", "role: assistant\ncontent: Hi!"])
        tokenizer.apply_chat_template(conversation=messages,
                                      tokenize=False,
                                      add_generation_prompt=True)
        for messages in list_of_messages
    ]

    # Log the input prompts
    for messages, full_prompt in zip(list_of_messages, list_of_texts):
        print("\n=== Input Prompt ===")
        print(full_prompt)
        print("==================\n")

    # Generate messages using the VLLM model with the list of texts and sampling parameters
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )

    # Process and log the outputs
    for messages, single_request_output in zip(list_of_messages,
                                               request_output):
        response_text = single_request_output.outputs[0].text

        print("\n=== Model Response ===")
        print(response_text)
        print("=====================\n")

        # Log to file
        log_model_interaction(messages, response_text)

        messages.append({'role': 'assistant', 'content': response_text})

    return list_of_messages

In [8]:
# Filter messages that contain boxed text and extract the boxed text as the answer
def batch_message_filter(
        list_of_messages) -> tuple[list[list[dict]], list[str]]:
    extracted_answers = []
    list_of_messages_to_keep = []
    for messages in list_of_messages:
        answer = extract_boxed_text(messages[-1]['content'])
        if answer:
            extracted_answers.append(answer)
        else:
            list_of_messages_to_keep.append(messages)
    return list_of_messages_to_keep, extracted_answers

In [9]:
# Execute Python code in messages and return the output as a string (e.g., "Hello, world!")
# or an empty string if no output is generated (e.g., if the input is empty)
# or an error occurs (e.g., syntax error) during code execution (e.g., "SyntaxError: invalid syntax")
# or if the code execution times out (e.g., "Execution timed out after 5 seconds.")
# or if the code execution fails (e.g., "Execution failed with exit code 1.")
# or if the code execution is successful but no output is generated (e.g., "Execution successful but no output.")
# or if the code execution is successful but the output is empty (e.g., "Execution successful but empty output.")
# or if the code execution is successful but the output is too long (e.g., "Execution successful but output is too long.")
# or if the code execution is successful but the output is too short (e.g., "Execution successful but output is too short.")
# or if the code execution is successful but the output is invalid (e.g., "Execution successful but invalid output.")
# or if the code execution is successful but the output is not a string (e.g., "Execution successful but output is not a string.")
# or if the code execution is successful but the output is not a valid answer
def batch_message_execute(list_of_messages) -> list[list[dict]]:
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'])
        python_code = process_python_code(python_code)
        # print('\n\n' + python_code + '\n\n')
        try:
            print('c', end='')
            is_successful, output = PythonREPL()(python_code)
            if is_successful:
                print('o', end='')
            else:
                print('e', end='')
        except Exception as e:
            print('f', end='')
            output = str(e)
        print(python_code)
        print()
        print(output)
        print("\n\n")
        messages.append({
            'role': 'user',
            'content': "```output\n" + output + "\n```"
        })
    print()
    return list_of_messages

In [ ]:
def create_starter_messages(question, index):
    cycle_size = 2
    if False:
        pass
    elif index % cycle_size == 1:
        # https://github.com/QwenLM/Qwen2.5-Math?tab=readme-ov-file#-hugging-face-transformers
        # CoT
        return [{
            "role":
            "system",
            "content":
            "Please reason step by step, and put your final answer within \\boxed{}."
        }, {
            "role":
            "user",
            "content":
            question + "\n\nBegin your answer by importing math and sympy."
        }]
    else:
        # https://github.com/QwenLM/Qwen2.5-Math?tab=readme-ov-file#-hugging-face-transformers
        # TIR
        return [{
            "role": 
            "system",
            "content":
            "Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."
        }, {
            "role":
            "user",
            "content":
            question + "\n\nBegin your answer by importing math and sympy."
        }]

In [11]:
"""
Dynamic Batch Size Optimization
-----------------------------
The following implementation replaces the static batch size of 32 with a dynamic batch sizing mechanism.

Rationale:
- Fixed batch sizes can be suboptimal as GPU memory availability varies during execution
- Large static batch sizes may cause OOM errors or memory fragmentation
- Small static batch sizes may underutilize available resources
- Memory requirements per conversation can vary based on problem complexity

Benefits:
- Adaptive resource utilization based on real-time GPU memory availability
- Reduced risk of OOM errors during long running sessions
- Better throughput by maximizing parallel processing when possible
- More resilient to varying problem complexities

Implementation notes:
- Uses 80% of available GPU memory to leave headroom for fluctuations
- Sets reasonable min/max bounds (8-48) to maintain stability
- Falls back to conservative batch size (16) if memory detection fails
- Considers multi-GPU setups by checking all available devices

Last modified: November 2024
"""


def get_optimal_batch_size():
    """
    Dynamically determine the optimal batch size for parallel message processing
    based on available GPU memory across all devices.
    
    The function:
    1. Checks available memory across all GPU devices
    2. Calculates safe batch size using conservative memory estimates
    3. Applies bounds to ensure stable execution
    
    Returns:
        int: Optimal batch size between 8 and 48
        
    Note:
        - Assumes ~0.5GB memory usage per conversation
        - Uses 80% of available memory as safety margin
        - Falls back to batch size 16 if memory detection fails
    """
    try:
        # Get available GPU memory
        gpu_memory = []
        for i in range(torch.cuda.device_count()):
            total_memory = torch.cuda.get_device_properties(i).total_memory
            allocated_memory = torch.cuda.memory_allocated(i)
            free_memory = total_memory - allocated_memory
            gpu_memory.append(free_memory)

        # Use the minimum free memory across all GPUs
        min_free_memory = min(gpu_memory)

        # Calculate batch size based on free memory
        # Conservative estimate: assume each conversation requires about 0.5GB
        memory_per_conversation = 0.5 * (1024**3)  # 0.5GB in bytes
        optimal_batch_size = int(
            (min_free_memory * 0.8) /
            memory_per_conversation)  # Use 80% of free memory

        # Ensure batch size is within reasonable bounds
        optimal_batch_size = max(8, min(48, optimal_batch_size))

        return optimal_batch_size
    except Exception as e:
        print(f"Error calculating batch size: {e}")
        return 16  # Default fallback batch size

In [ ]:
def predict_for_question(question: str) -> int:
    print("\n" + "=" * 80 + "\nNEW QUESTION\n" + "=" * 80)

    import os
    # only run this code if it's not a competition rerun
    if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        # only run this code if the question is not the example question from the competition page (to avoid wasting time)
        if question != "Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?":
            return 210
    # check if the time limit has been reached
    if time.time() > cutoff_time:
        return 210

    question += "\nIf the final answer is a number larger than 1 million, take modulo 1000."
    print(question)

    # Dynamic batch size instead of fixed 32 for parallel message processing (e.g., for tensor parallelism) based on available GPU memory across all devices (e.g., 8-48 conversations in parallel)
    batch_size = get_optimal_batch_size()
    list_of_messages = [
        create_starter_messages(question, index) for index in range(batch_size)
    ]

    all_extracted_answers = []
    # 4 rounds of message generation, filtering, and execution
    for _ in range(4):
        # Generate messages using the VLLM model
        list_of_messages = batch_message_generate(list_of_messages)
        # Filter messages that contain boxed text and extract the boxed text as the answer
        list_of_messages, extracted_answers = batch_message_filter(
            list_of_messages)
        # Extend the list of extracted answers
        all_extracted_answers.extend(extracted_answers)
        if not list_of_messages:
            break
        # Execute Python code in messages and return the output as a string
        list_of_messages = batch_message_execute(list_of_messages)

    print(all_extracted_answers)
    answer = select_answer(all_extracted_answers)
    print(answer)

    print("\n\n")
    return answer

In [13]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame,
            question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    # get the first element of the DataFrame
    id_ = id_.item(0)
    print("------")
    print(id_)

    # get the first element of the DataFrame
    question = question.item(0)
    answer = predict_for_question(question)
    print(question)
    print("------\n\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [14]:
# predict_for_question("Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?")

In [15]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [16]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(
    predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway((
        #             '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv',
        'reference.csv', ))

------
bbd91e
Alice writes all positive integers from $1$ to $n$ on the board for some positive integer $n \geq 11$. Bob then erases ten of them. The mean of the remaining numbers is $3000/37$. The sum of the numbers Bob erased is $S$. What is the remainder when $n \times S$ is divided by $997$?
------



------
057f8a
Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?
------



------
480182
Let $ABC$ be a triangle with $BC=108$, $CA=126$, and $AB=39$. Point $X$ lies on segment $AC$ such that $BX$ bisects $\angle CBA$. Let $\omega$ be the circumcircle of triangle $ABX$. Let $Y$ be a point on $\omega$ different from $X$ such that $CX

Processed prompts: 100%|██████████| 8/8 [02:33<00:00, 19.24s/it, est. speed input: 5.95 toks/s, output: 105.00 toks/s]


coimport math
import numpy as np
import sympy as sp





coimport math
import numpy as np
import sympy as sp





coimport math
import numpy as np
import sympy as sp
import sympy as sp

# Define the sides a and b
a = sp.Symbol('a', positive=True, real=True)

try:
    print(f"a={str(a)[:100]}")
except:
    pass

b = sp.Symbol('b', positive=True, real=True)

try:
    print(f"b={str(b)[:100]}")
except:
    pass


# Given side c and circumradius R
c = 120

try:
    print(f"c={str(c)[:100]}")
except:
    pass

R = 100

try:
    print(f"R={str(R)[:100]}")
except:
    pass


# Expression for area K in terms of sides and circumradius
K = (a * b * c) / (4 * R)

try:
    print(f"K={str(K)[:100]}")
except:
    pass


# Expression for area K in terms of base and height
CD = sp.Symbol('CD')

try:
    print(f"CD={str(CD)[:100]}")
except:
    pass

K_CD = (1/2) * c * CD

try:
    print(f"K_CD={str(K_CD)[:100]}")
except:
    pass


# Equate the two expressions for K
equation = sp.Eq(K, K_CD)

try:
   

Processed prompts: 100%|██████████| 8/8 [02:52<00:00, 21.51s/it, est. speed input: 103.06 toks/s, output: 66.64 toks/s]

['6', '120', '39.6', '23', '36', '249', '96', '1']
36



Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?
------



------
a1d40b
The Fibonacci numbers are defined as follows: $F_0 = 0$, $F_1 = 1$, and $F_{n+1} = F_n + F_{n-1}$ for $n \geq 1$. There are $N$ positive integers $n$ strictly less than $10^{101}$ such that $n^2 + (n+1)^2$ is a multiple of 5 but $F_{n-1}^2 + F_n^2$ is not. How many prime factors does $N$ have, counted with multiplicity?
------



------
1fce4b
Find the three-digit number $n$ such that writing any other three-digit number $10^{2024}$ times in a row and $10^{2024}+2$ times in a row results in two numbers divisible by $n$.
------



------
192e23
Fred and George take part in a tennis tournament with $4046$ other players. In each round, the players are paired into $2024$ matches. How many ways are there to arrange the fir

In [17]:
"""
import gc  # garbage collector

# clean memory (RAM and GPU memory) to avoid memory leaks and out-of-memory errors (e.g., due to PyTorch tensor parallelism)
# and improve performance (e.g., by reducing memory fragmentation) and stability (e.g., by avoiding memory leaks) of the VLLM model
def clean_memory(deep=False):
    gc.collect()  # garbage collector (RAM)
    if deep:
        # memory allocator (RAM) for PyTorch tensors
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    # memory allocator (GPU) for PyTorch tensors
    torch.cuda.empty_cache()


# delete the VLLM model to free up GPU memory
del llm

# clean memory (RAM and GPU memory) to avoid memory leaks and out-of-memory errors
clean_memory(deep=True)
"""

'\nimport gc  # garbage collector\n\n# clean memory (RAM and GPU memory) to avoid memory leaks and out-of-memory errors (e.g., due to PyTorch tensor parallelism)\n# and improve performance (e.g., by reducing memory fragmentation) and stability (e.g., by avoiding memory leaks) of the VLLM model\ndef clean_memory(deep=False):\n    gc.collect()  # garbage collector (RAM)\n    if deep:\n        # memory allocator (RAM) for PyTorch tensors\n        ctypes.CDLL("libc.so.6").malloc_trim(0)\n    # memory allocator (GPU) for PyTorch tensors\n    torch.cuda.empty_cache()\n\n\n# delete the VLLM model to free up GPU memory\ndel llm\n\n# clean memory (RAM and GPU memory) to avoid memory leaks and out-of-memory errors\nclean_memory(deep=True)\n'

In [ ]:
# Modified file handling code
def save_and_display_files():
    """Save files and create reliable download links"""
    import os
    from IPython.display import HTML, FileLink, display
    import shutil

    # Ensure we're in the Kaggle working directory
    working_dir = "/kaggle/working"
    if not os.path.exists(working_dir):
        print("Error: Not running in Kaggle environment")
        return

    # Function to safely handle file copy
    def safe_copy(src, dst):
        try:
            shutil.copy2(src, dst)
            return True
        except Exception as e:
            print(f"Error copying {src}: {e}")
            return False

    # Create a new directory for our outputs
    output_dir = os.path.join(working_dir, "outputs")
    os.makedirs(output_dir, exist_ok=True)

    # List and copy all relevant files
    files_copied = []
    for root, _, files in os.walk(working_dir):
        for filename in files:
            # Skip the outputs directory itself and any temp files
            if "outputs" in root or filename.startswith('.'):
                continue

            src_path = os.path.join(root, filename)
            dst_path = os.path.join(output_dir, filename)

            if safe_copy(src_path, dst_path):
                files_copied.append(filename)

    # Create HTML display for available files
    if files_copied:
        print("\nFiles available for download:")
        for filename in sorted(files_copied):
            try:
                # Create FileLink for each file
                file_path = os.path.join("outputs", filename)
                display(FileLink(file_path,
                                 result_html_prefix=f"{filename}: "))
            except Exception as e:
                print(f"Error creating link for {filename}: {e}")
    else:
        print("No files were found to copy")

    # Create zip file of all outputs
    try:
        zip_path = os.path.join(working_dir, "outputs.zip")
        shutil.make_archive(os.path.join(working_dir, "outputs"), 'zip',
                            output_dir)
        print("\nAll files are also available in a single zip:")
        display(
            FileLink("outputs.zip", result_html_prefix="Download all files: "))
    except Exception as e:
        print(f"Error creating zip file: {e}")


# Run the function
save_and_display_files()